## Import Packages

In [1]:
import pandas as pd
import numpy as np
import os
import time

from PIL import Image

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


## Load Dataset

In [2]:
# Set data folders

train_img_path = "./train/"
test_img_path = "./test/"
labels_csv_path = "train.csv"
sample_sub_path = "sample_submission.csv"

## Make Dataset class

In [5]:
# 파이토치 Dataset 클래스 만들기
# 여기엔 __len__, __getitem__ 펑션 포함되어 있어야 함
# 추가적으로 이미지를 transform할거라면 transform에 대한 펑션도 미리 정의해야 함

class CactusDataset(Dataset):
    
    def __init__(self, img_dir, dataframe, transform=None):
        self.labels_frame = dataframe
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels_frame)
    
    def __getitem(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_frame[idx])
        image = Image.open(img_name)
        label = self.labels_frame.has_cactus[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return [image, label]

In [7]:
# Data transform에 대한 부분

train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [9]:
# 데이터를 불러와서 5%는 테스트셋으로 남겨준다.
dframe = pd.read_csv(labels_csv_path)
cut = int(len(dframe) * 0.95)
train, test = np.split(dframe, [cut], axis=0)
test = test.reset_index(drop=True) # 0부터 시작하는 새로운 인덱스 적용

In [16]:
# 이건 train 셋에서 5% 남겨놓은 테스트셋이므로...그래도 둡니다.
train_ds = CactusDataset(train_img_path, train, train_transforms)
test_ds = CactusDataset(train_img_path, test, test_transforms)
datasets = {"train" : train_ds, "test" : test_ds}

In [17]:
trainloader = DataLoader(train_ds, batch_size=32, shuffle=True)
testloader = DataLoader(test_ds, batch_size=4, shuffle=True)

In [19]:
epochs = 20
batch_size = 128
learning_rate = 0.003
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [8]:
# torchvision 모듈에서 pretrained된 모델들을 로드해서 이용할 수 있음
# 여기에서는 vgg16을 로드하고, pretrained weight를 모두 가지고 온다.

model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/dongheelee/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [01:31<00:00, 6.02MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [29]:
# pretrained 된 모델의 마지막 classifier부분을 우리 목적에 맞게 변형해준다.

for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 12000)),
    ('dr1', nn.Dropout(p=0.3)),
    ('bn1', nn.BatchNorm1d(num_features=12000)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(12000, 1000)),
    ('dr2', nn.Dropout(p=0.3)),
    ('bn2', nn.BatchNorm1d(num_features=1000)),
    ('relu2', nn.ReLU()),
    ('fc3', nn.Linear(1000, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

In [31]:
# criterion, loss 정의
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

In [ ]:
#train model
model.to(device)

steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    # calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            running_loss = 0
            model.train()